In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
from tqdm import tqdm
warnings.filterwarnings(action='ignore') 
# import seaborn as sns

In [ ]:
!pip install catboost
!pip install ipywidgets
!pip install shap
#!pip install sklearn
!jupyter nbextension enable --py widgetsnbextension

In [ ]:
from google.colab import drive
drive.mount('/gdrive')

In [ ]:
base_path = "/gdrive/My Drive/Colab Notebooks/KNOW_data/"

In [ ]:
from glob import glob
train = [pd.read_csv(path) for path in sorted(glob(base_path + 'train/*.csv'))]


In [ ]:
def none_to_zero(data):
    for df in data:
        df.replace(' ', np.nan, inplace=True)
    return data
train = none_to_zero(train)

In [ ]:
test = [pd.read_csv(path) for path in sorted(glob(base_path + 'test/*.csv'))]
test = none_to_zero(test)

In [ ]:
# def modify_bq4_to_1c(data):
#     for index in data[(data['bq4_1c'] == '1') | (data['bq4_1c'] == '2')].index:
#         if data.loc[index, ['bq4']].isnull()[0]:
#             # 'bq4' 칼럼에 있는 값이 결측치 값일 경우
#             # 칼럼에 해당하는 값이 결측치 값일 경우는 'bq4'에 해당하는 갑셍는 '2'로 설정
#             # 나머지 'bq4_1a' ~ 'bq4_1c'에 해당하는 값에는 '없음' 으로 바꿔 준다.
            
#             # 해당하는 자격증이 없기 때문에 '2'로 바꿔준다.
#             data.loc[index, 'bq4'] = '2'
            
#             # 'bq4_1a' 칼럼을 '없음' 으로 바꿔준다.
#             data.loc[index, 'bq4_1a'] = '없음'
            
#             # 'bq4_1b' 칼럼을 '없음' 으로 바꿔준다.
#             data.loc[index, 'bq4_1b'] = '없음'
            
#             # 'bq4_1c' 칼럼을 '없음' 으로 바꿔준다.
#             data.loc[index, 'bq4_1c'] = '없음'
#             pass
#         else:
#             # 'bq4' 칼럼에 있는 값이 결측치 값이 아닐 경우
#             if (data.loc[index, ['bq4_1a']].isnull()[0]) & (data.loc[index, ['bq4_1b']].isnull()[0]):
#                 # 'bq4_1a'와 'bq4_1b' 값이 모두 결측치 값일 경우이다.
#                 data.loc[index, 'bq4_1a'] = data.loc[index, 'bq4']
#                 data.loc[index, 'bq4'] = '1'
#                 data.loc[index, 'bq4_1c'] = np.nan
#                 pass
#             elif data.loc[index, ['bq4_1b']].isnull()[0]:
#                 data.loc[index, 'bq4_1c'] = data.loc[index, 'bq4_1b']
#                 data.loc[index, 'bq4_1b'] = data.loc[index, 'bq4_1a']
#                 data.loc[index, 'bq4_1a'] = data.loc[index, 'bq4']
#                 data.loc[index, 'bq4'] = '1'
#                 pass
#             else:
#                 data.loc[index, 'bq4_1c'] = data.loc[index, 'bq4_1b']
#                 data.loc[index, 'bq4_1b'] = data.loc[index, 'bq4_1a']
#                 data.loc[index, 'bq4_1a'] = data.loc[index, 'bq4']
#                 data.loc[index, 'bq4'] = '1'

# def preprocessing_2018(train_2018, test_2018):
#   have_nan_columns = []
#   for index, value, tindex, tvalue in zip(train_2018.isnull().sum().index, train_2018.isnull().sum(), test_2018.isnull().sum().index, test_2018.isnull().sum()):
#       print(f"{index} : {value} / {tindex} : {tvalue} / {value + tvalue}")
#       if (value > 0) | (tvalue > 0):
#           have_nan_columns.append(index)

#   train_indexs = train_2018[((train_2018['bq4_1c'] == '1') | (train_2018['bq4_1c'] == '2')) & (train_2018['bq4'].notnull())][['bq4', 'bq4_1a', 'bq4_1b', 'bq4_1c']].index
#   test_indexs = test_2018[((test_2018['bq4_1c'] == '1') | (test_2018['bq4_1c'] == '2')) & (test_2018['bq4'].notnull())][['bq4', 'bq4_1a', 'bq4_1b', 'bq4_1c']].index

#   modify_bq4_to_1c(train_2018)            
#   modify_bq4_to_1c(test_2018)

#   for col in ['bq4_1a', 'bq4_1b', 'bq4_1c']:
#     train_2018[col].replace(np.nan, '없음', inplace = True)
#     test_2018[col].replace(np.nan, '없음', inplace = True)

#   # 'bq5_2' 칼럼에서 숫자를 같고 있는 경우에도 'bq5'와 'bq5_1'에서 결측치 값이 아닌 경우가 존재한다.
#   train_2018[(train_2018['bq5_2'] == '7')][['bq5', 'bq5_1', 'bq5_2']]

#   modify_train = []
#   modify_test = []

#   # bq5_2에서 숫자 형태를 갖고 있는 모든 사람들을 뽑아낸다.
#   for index in train_2018.index:
#       # 결측치 값은 넘긴다.
#       if train_2018.loc[index, ['bq5_2']].isnull()[0]:
#           continue
#       # 숫자 형태는 아스키 코드 값이 48이상이고 57이하의 값들이다.
#       # 아스키 코드를 통해서 한글자로 표현된 문장들은 건너뛴다.
#       elif len(train_2018.loc[index, 'bq5_2']) == 1:
#           if ord(train_2018.loc[index, 'bq5_2']) >= 48 and ord(train_2018.loc[index, 'bq5_2']) <= 57:
#               modify_train.append(index)
#       # 이외의 값들도 건너뛴다.
#       else:
#           continue
          
#   # bq5_2에서 숫자 형태를 갖고 있는 모든 사람들을 뽑아낸다.
#   for index in test_2018.index:
#       # 결측치 값은 넘긴다.
#       if test_2018.loc[index, ['bq5_2']].isnull()[0]:
#           continue
#       # 숫자 형태는 아스키 코드 값이 48이상이고 57이하의 값들이다.
#       # 아스키 코드를 통해서 한글자로 표현된 문장들은 건너뛴다.
#       elif len(test_2018.loc[index, 'bq5_2']) == 1:
#           if ord(test_2018.loc[index, 'bq5_2']) >= 48 and ord(test_2018.loc[index, 'bq5_2']) <= 57:
#               modify_test.append(index)
#       # 이외의 값들도 건너뛴다.
#       else:
#           continue       
#   for index in modify_train:
#     # 5-1번 문항에 번호가 6개가 있으므로 6개에 해당해야 한다.
#     if train_2018.loc[index, 'bq5'] in ['1', '2', '3', '4', '5', '6'] and train_2018.loc[index, ['bq5_1']].notnull()[0]:
#         train_2018.loc[index, 'bq5_2'] = train_2018.loc[index, 'bq5_1']
#         train_2018.loc[index, 'bq5_1'] = train_2018.loc[index, 'bq5']
#         train_2018.loc[index, 'bq5'] = '1'
#     else:
#         train_2018.loc[index, 'bq5_2'] = np.nan
#         train_2018.loc[index, 'bq5_1'] = np.nan
#         train_2018.loc[index, 'bq5'] = '2'

#   for index in modify_test:
#     # 5-1번 문항에 번호가 6개가 있으므로 6개에 해당해야 한다.
#     if test_2018.loc[index, 'bq5'] in ['1', '2', '3', '4', '5', '6'] and test_2018.loc[index, ['bq5_1']].notnull()[0]:
#         test_2018.loc[index, 'bq5_2'] = test_2018.loc[index, 'bq5_1']
#         test_2018.loc[index, 'bq5_1'] = test_2018.loc[index, 'bq5']
#         test_2018.loc[index, 'bq5'] = '1'
#     else:
#         test_2018.loc[index, 'bq5_2'] = np.nan
#         test_2018.loc[index, 'bq5_1'] = np.nan
#         test_2018.loc[index, 'bq5'] = '2'

#   for col in ['bq5', 'bq5_1', 'bq5_2']:
#     train_2018[col].replace(np.nan, '없음', inplace = True)
#     test_2018[col].replace(np.nan, '없음', inplace = True)
#   train_2018.loc[(train_2018['bq12_1'].isnull())]['bq12_1'] = '9'

#   train_indexs = train_2018[(train_2018['bq12_2'].isnull())].index
#   for index in train_2018[(train_2018['bq12_2'].isnull())].index:
#     # 산업 유형
#     industry_type = train_2018.loc[index, 'bq1']
#     # 근무업체 유형
#     work_company_type = train_2018.loc[index, 'bq2']
#     # 교육수준
#     education_level = train_2018.loc[index, 'bq7']
#     # 사회적 평판_1
#     social_reputation_1 = train_2018.loc[index, 'bq8_1']
#     # 사회적 평판_2
#     social_reputation_2 = train_2018.loc[index, 'bq8_2']
#     # 직업 안정성
#     job_security = train_2018.loc[index, 'bq9']
#     # 근무조건
#     working_conditions = train_2018.loc[index, 'bq11']
#     # 발전가능성
#     advancement  = train_2018.loc[index, 'bq10']
    
#     # 승진에 대한 좋고 나쁨은 '산업 유형', '근무업체 유형', '교육수준', '직업 안정성', '근무조건'에서 기인한다고 추측 했습니다.
#     # 교육수준이 높을 수록 좋은 회사에 들어갈 확률이 높습니다. 
#     # 따라서, 좋은 회사일 수록 승진의 기회가 좁아지는 것은 올바른 추측이라고 생각합니다.
#     # 사회적 평판과 승진과는 별다른 인과관계가 없다고 생각했기 때문에 이 또한 승진에 대한 추측에서 제외했습니다.
#     if len(train_2018[(train_2018['bq12_2'].notnull()) & (train_2018['bq1'] == industry_type) & (train_2018['bq2'] == work_company_type) & (train_2018['bq9'] == job_security) & (train_2018['bq11'] == working_conditions)]) >= 1:
#         # '산업 유형', '근무업체 유형', '직업 안정성', '근무조건'
#         train_2018.loc[index, 'bq12_2'] = train_2018.loc[(train_2018['bq1'] == industry_type) & (train_2018['bq2'] == work_company_type) & (train_2018['bq9'] == job_security) & (train_2018['bq11'] == working_conditions), 'bq12_2'].value_counts().index[0]
#     elif len(train_2018[(train_2018['bq12_2'].notnull()) & (train_2018['bq1'] == industry_type) & (train_2018['bq2'] == work_company_type) & (train_2018['bq9'] == job_security)]) >= 1:
#         # '산업 유형', '근무업체 유형', '직업 안정성'
#         train_2018.loc[index, 'bq12_2'] = train_2018.loc[(train_2018['bq1'] == industry_type) & (train_2018['bq2'] == work_company_type) & (train_2018['bq9'] == job_security), 'bq12_2'].value_counts().index[0]
#     elif len(train_2018[(train_2018['bq12_2'].notnull()) & (train_2018['bq9'] == job_security) & (train_2018['bq10'] == advancement)]) >= 1:
#         # '직업 안정성', '발전 가능성'
#         train_2018.loc[index, 'bq12_2'] = train_2018[(train_2018['bq9'] == job_security) & (train_2018['bq10'] == advancement)]['bq12_2'].value_counts().index[0]
#     elif len(train_2018[(train_2018['bq12_2'].notnull()) & (train_2018['bq1'] == industry_type) & (train_2018['bq9'] == job_security)]) >= 1:
#         # '산업 유형', '직업 안정성'
#         train_2018.loc[index, 'bq12_2'] = train_2018[(train_2018['bq1'] == industry_type) & (train_2018['bq9'] == job_security)]['bq12_2'].value_counts().index[0]
#     else:
#         # 학습 데이터 상에서는 위에 조건에 다 만족한다.
#         pass

#   test_indexs = test_2018[(test_2018['bq12_2'].isnull())].index
#   for index in test_2018[(test_2018['bq12_2'].isnull())].index:
#     # 산업 유형
#     industry_type = test_2018.loc[index, 'bq1']
#     # 근무업체 유형
#     work_company_type = test_2018.loc[index, 'bq2']
#     # 교육수준
#     education_level = test_2018.loc[index, 'bq7']
#     # 사회적 평판_1
#     social_reputation_1 = test_2018.loc[index, 'bq8_1']
#     # 사회적 평판_2
#     social_reputation_2 = test_2018.loc[index, 'bq8_2']
#     # 직업 안정성
#     job_security = test_2018.loc[index, 'bq9']
#     # 근무조건
#     working_conditions = test_2018.loc[index, 'bq11']
#     # 발전가능성
#     advancement  = test_2018.loc[index, 'bq10']
    
#     # 승진에 대한 좋고 나쁨은 '산업 유형', '근무업체 유형', '교육수준', 직업 안정성', '근무조건'에서 기인한다고 추측 했습니다.
#     # 교육수준이 높을 수록 좋은 회사에 들어갈 확률이 높습니다. 
#     # 따라서, 좋은 회사일 수록 승진의 기회가 좁아지는 것은 올바른 추측이라고 생각합니다.
#     # 사회적 평판과 승진과는 별다른 인과관계가 없다고 생각했기 때문에 이 또한 승진에 대한 추측에서 제외했습니다.
#     if len(test_2018[(test_2018['bq12_2'].notnull()) & (test_2018['bq1'] == industry_type) & (test_2018['bq2'] == work_company_type) & (test_2018['bq9'] == job_security) & (test_2018['bq11'] == working_conditions)]) >= 1:
#         # '산업 유형', '근무업체 유형', '직업 안정성', '근무조건'
#         test_2018.loc[index, 'bq12_2'] = test_2018.loc[(test_2018['bq1'] == industry_type) & (test_2018['bq2'] == work_company_type) & (test_2018['bq9'] == job_security) & (test_2018['bq11'] == working_conditions), 'bq12_2'].value_counts().index[0]
#     elif len(test_2018[(test_2018['bq12_2'].notnull()) & (test_2018['bq1'] == industry_type) & (test_2018['bq2'] == work_company_type) & (test_2018['bq9'] == job_security)]) >= 1:
#         # '산업 유형', '근무업체 유형', '직업 안정성'
#         test_2018.loc[index, 'bq12_2'] = test_2018.loc[(test_2018['bq1'] == industry_type) & (test_2018['bq2'] == work_company_type) & (test_2018['bq9'] == job_security), 'bq12_2'].value_counts().index[0]
#     elif len(test_2018[(test_2018['bq12_2'].notnull()) & (test_2018['bq9'] == job_security) & (test_2018['bq10'] == advancement)]) >= 1:
#         # '직업 안정성', '발전 가능성'
#         test_2018.loc[index, 'bq12_2'] = test_2018[(test_2018['bq9'] == job_security) & (test_2018['bq10'] == advancement)]['bq12_2'].value_counts().index[0]
#     elif len(test_2018[(test_2018['bq12_2'].notnull()) & (test_2018['bq1'] == industry_type) & (test_2018['bq9'] == job_security)]) >= 1:
#         # '산업 유형', '직업 안정성'
#         test_2018.loc[index, 'bq12_2'] = test_2018[(test_2018['bq1'] == industry_type) & (test_2018['bq9'] == job_security)]['bq12_2'].value_counts().index[0]
#     else:
#         # 테스트 데이터 상애서도 위를 다 만족한다.
#         pass
#     train_indexs = train_2018[(train_2018['bq12_3'].isnull())].index
#   for index in train_2018[(train_2018['bq12_3'].isnull())].index:
#     # 산업 유형
#     industry_type = train_2018.loc[index, 'bq1']
#     # 근무업체 유형
#     work_company_type = train_2018.loc[index, 'bq2']
#     # 교육수준
#     education_level = train_2018.loc[index, 'bq7']
#     # 사회적 평판_1
#     social_reputation_1 = train_2018.loc[index, 'bq8_1']
#     # 사회적 평판_2
#     social_reputation_2 = train_2018.loc[index, 'bq8_2']
#     # 직업 안정성
#     job_security = train_2018.loc[index, 'bq9']
#     # 근무조건
#     working_conditions = train_2018.loc[index, 'bq11']
#     # 발전가능성
#     advancement  = train_2018.loc[index, 'bq10']
    
#     # 상사에 대한 좋고 나쁨은 '산업 유형', '근무업체 유형', '근무조건' 에 크게 영향을 받는다고 생각한다.
#     # 상사가 나쁜 회사일 수록 '귀하의 자녀가 귀하의 현재 직업과 동일한 작업을 원한다' 라는 질문에서 나쁘게 선택했을 가능성이 크다.
#     # 부모 되는 입장에서 자식에게 나쁜 것을 물려주고 싶은 부모는 없다고 생각하기 때문이다.
#     # 교육수준은 상사에 대한 평가에서는 제외하도록 하겠다. 
#     # 교육수준이 높다고 해서 무조건 적으로 좋은 상사라고 생각하지 않는다. 
#     # 나쁜 사람이 나쁜 것이지 그것이 교육 수준하고는 무관하기 때문이다. 
    
#     # # bq1, bq2, bq8_2, bq11
#     # if len(train_2018[(train_2018['bq12_3'].notnull()) & (train_2018['bq1'] == industry_type) & (train_2018['bq2'] == work_company_type) & (train_2018['bq8_2'] == social_reputation_2) & (train_2018['bq11'] == working_conditions)]) >= 1:
#     #     # '산업 유형', '근무업체 유형', '사회적 평판', '근무조건'
#     #     train_2018.loc[index, 'bq12_3'] = train_2018[(train_2018['bq12_3'].notnull()) & (train_2018['bq1'] == industry_type) & (train_2018['bq2'] == work_company_type) & (train_2018['bq8_2'] == social_reputation_2) & (train_2018['bq11'] == working_conditions)]['bq12_3'].value_counts().index[0]
#     # elif len(train_2018[(train_2018['bq12_3'].notnull()) & (train_2018['bq1'] == industry_type) & (train_2018['bq2'] == work_company_type) & (train_2018['bq8_2'] == social_reputation_2)]) >= 1:
#     #     # '산업 유형', '근무업체 유형', '사회적 평판'
#     #     train_2018.loc[index, 'bq12_3'] = train_2018[(train_2018['bq12_3'].notnull()) & (train_2018['bq1'] == industry_type) & (train_2018['bq2'] == work_company_type) & (train_2018['bq8_2'] == social_reputation_2)]['bq12_3'].value_counts().index[0]
#     # elif len(train_2018[(train_2018['bq12_3'].notnull()) & (train_2018['bq1'] == industry_type) & (train_2018['bq2'] == work_company_type) & (train_2018['bq11'] == working_conditions)]) >= 1:
#     #     # '산업 유형', '근무업체 유형', '근무조건'
#     #     train_2018.loc[index, 'bq12_3'] = train_2018[(train_2018['bq12_3'].notnull()) & (train_2018['bq1'] == industry_type) & (train_2018['bq2'] == work_company_type) & (train_2018['bq11'] == working_conditions)]['bq12_3'].value_counts().index[0]
#     # elif len(train_2018[(train_2018['bq12_3'].notnull()) & (train_2018['bq1'] == industry_type) & (train_2018['bq2'] == work_company_type)]) >= 1:
#     #     #  '산업 유형', '근무업체 유형'
#     #     train_2018.loc[index, 'bq12_3'] = train_2018[(train_2018['bq12_3'].notnull()) & (train_2018['bq1'] == industry_type) & (train_2018['bq2'] == work_company_type)]['bq12_3'].value_counts().index[0]
#     # elif len(train_2018[(train_2018['bq12_3'].notnull()) & (train_2018['bq1'] == industry_type) & (train_2018['bq8_2'] == social_reputation_2)]) >= 1:
#     #     # '산업 유형', '사회적 평판'
#     #     train_2018.loc[index, 'bq12_3'] = train_2018[(train_2018['bq12_3'].notnull()) & (train_2018['bq1'] == industry_type) & (train_2018['bq8_2'] == social_reputation_2)]['bq12_3'].value_counts().index[0]
#     # else:
#     #     # 학습 데이터에서는 위의 가정에 모두 속해있다.
#     #     pass
#   # test_indexs = test_2018[(test_2018['bq12_3'].isnull())].index
#   # for index in test_2018[(test_2018['bq12_3'].isnull())].index:
#   #   # 산업 유형
#   #   industry_type = test_2018.loc[index, 'bq1']
#   #   # 근무업체 유형
#   #   work_company_type = test_2018.loc[index, 'bq2']
#   #   # 교육수준
#   #   education_level = test_2018.loc[index, 'bq7']
#   #   # 사회적 평판_1
#   #   social_reputation_1 = test_2018.loc[index, 'bq8_1']
#   #   # 사회적 평판_2
#   #   social_reputation_2 = test_2018.loc[index, 'bq8_2']
#   #   # 직업 안정성
#   #   job_security = test_2018.loc[index, 'bq9']
#   #   # 근무조건
#   #   working_conditions = test_2018.loc[index, 'bq11']
#   #   # 발전가능성
#   #   advancement  = test_2018.loc[index, 'bq10']
    
#     # 상사에 대한 좋고 나쁨은 '산업 유형', '근무업체 유형', '근무조건' 에 크게 영향을 받는다고 생각한다.
#     # 상사가 나쁜 회사일 수록 '귀하의 자녀가 귀하의 현재 직업과 동일한 작업을 원한다' 라는 질문에서 나쁘게 선택했을 가능성이 크다.
#     # 부모 되는 입장에서 자식에게 나쁜 것을 물려주고 싶은 부모는 없다고 생각하기 때문이다.
#     # 교육수준은 상사에 대한 평가에서는 제외하도록 하겠다. 
#     # 교육수준이 높다고 해서 무조건 적으로 좋은 상사라고 생각하지 않는다. 
#     # 나쁜 사람이 나쁜 것이지 그것이 교육 수준하고는 무관하기 때문이다. 
    
#     # # bq1, bq2, bq8_2, bq11
#     # if len(test_2018[(test_2018['bq12_3'].notnull()) & (test_2018['bq1'] == industry_type) & (test_2018['bq2'] == work_company_type) & (test_2018['bq8_2'] == social_reputation_2) & (test_2018['bq11'] == working_conditions)]) >= 1:
#     #     # '산업 유형', '근무업체 유형', '사회적 평판', '근무조건'
#     #     test_2018.loc[index, 'bq12_3'] = test_2018[(test_2018['bq12_3'].notnull()) & (test_2018['bq1'] == industry_type) & (test_2018['bq2'] == work_company_type) & (test_2018['bq8_2'] == social_reputation_2) & (test_2018['bq11'] == working_conditions)]['bq12_3'].value_counts().index[0]
#     # elif len(test_2018[(test_2018['bq12_3'].notnull()) & (test_2018['bq1'] == industry_type) & (test_2018['bq2'] == work_company_type) & (test_2018['bq8_2'] == social_reputation_2)]) >= 1:
#     #     # '산업 유형', '근무업체 유형', '사회적 평판'
#     #     test_2018.loc[index, 'bq12_3'] = test_2018[(test_2018['bq12_3'].notnull()) & (test_2018['bq1'] == industry_type) & (test_2018['bq2'] == work_company_type) & (test_2018['bq8_2'] == social_reputation_2)]['bq12_3'].value_counts().index[0]
#     # elif len(test_2018[(test_2018['bq12_3'].notnull()) & (test_2018['bq1'] == industry_type) & (test_2018['bq2'] == work_company_type) & (test_2018['bq11'] == working_conditions)]) >= 1:
#     #     # '산업 유형', '근무업체 유형', '근무조건'
#     #     test_2018.loc[index, 'bq12_3'] = test_2018[(test_2018['bq12_3'].notnull()) & (test_2018['bq1'] == industry_type) & (test_2018['bq2'] == work_company_type) & (test_2018['bq11'] == working_conditions)]['bq12_3'].value_counts().index[0]
#     # elif len(test_2018[(test_2018['bq12_3'].notnull()) & (test_2018['bq1'] == industry_type) & (test_2018['bq2'] == work_company_type)]) >= 1:
#     #     #  '산업 유형', '근무업체 유형'
#     #     test_2018.loc[index, 'bq12_3'] = test_2018[(test_2018['bq12_3'].notnull()) & (test_2018['bq1'] == industry_type) & (test_2018['bq2'] == work_company_type)]['bq12_3'].value_counts().index[0]
#     # elif len(test_2018[(test_2018['bq12_3'].notnull()) & (test_2018['bq1'] == industry_type) & (test_2018['bq8_2'] == social_reputation_2)]) >= 1:
#     #     # '산업 유형', '사회적 평판'
#     #     test_2018.loc[index, 'bq12_3'] = test_2018[(test_2018['bq12_3'].notnull()) & (test_2018['bq1'] == industry_type) & (test_2018['bq8_2'] == social_reputation_2)]['bq12_3'].value_counts().index[0]
#     # else:
#     #     # 테스트 데이터에서는 위의 가정에 모두 속해있다.
#     #     pass

#   # train_indexs = train_2018[(train_2018['bq12_4'].isnull())].index
#   # for index in train_2018[(train_2018['bq12_4'].isnull())].index:
#   #   # 산업 유형
#   #   industry_type = train_2018.loc[index, 'bq1']
#   #   # 근무업체 유형
#   #   work_company_type = train_2018.loc[index, 'bq2']
#   #   # 교육수준
#   #   education_level = train_2018.loc[index, 'bq7']
#   #   # 사회적 평판_1
#   #   social_reputation_1 = train_2018.loc[index, 'bq8_1']
#   #   # 사회적 평판_2
#   #   social_reputation_2 = train_2018.loc[index, 'bq8_2']
#   #   # 직업 안정성
#   #   job_security = train_2018.loc[index, 'bq9']
#   #   # 근무조건
#   #   working_conditions = train_2018.loc[index, 'bq11']
#   #   # 발전가능성
#   #   advancement  = train_2018.loc[index, 'bq10']
    
#   #   # 동료에 대한 좋고 나쁨은 위의 상사와 비슷하다고 생각한다.
#   #   # 따라서 동일한 가정을 통해서 추측하도록 하겠다. 
#   #       # bq1, bq2, bq8_2, bq11
#   #   if len(train_2018[(train_2018['bq12_4'].notnull()) & (train_2018['bq1'] == industry_type) & (train_2018['bq2'] == work_company_type) & (train_2018['bq8_2'] == social_reputation_2) & (train_2018['bq11'] == working_conditions)]) >= 1:
#   #       # '산업 유형', '근무업체 유형', '사회적 평판', '근무조건'
#   #       train_2018.loc[index, 'bq12_4'] = train_2018[(train_2018['bq12_4'].notnull()) & (train_2018['bq1'] == industry_type) & (train_2018['bq2'] == work_company_type) & (train_2018['bq8_2'] == social_reputation_2) & (train_2018['bq11'] == working_conditions)]['bq12_4'].value_counts().index[0]
#   #   elif len(train_2018[(train_2018['bq12_4'].notnull()) & (train_2018['bq1'] == industry_type) & (train_2018['bq2'] == work_company_type) & (train_2018['bq8_2'] == social_reputation_2)]) >= 1:
#   #       # '산업 유형', '근무업체 유형', '사회적 평판'
#   #       train_2018.loc[index, 'bq12_4'] = train_2018[(train_2018['bq12_4'].notnull()) & (train_2018['bq1'] == industry_type) & (train_2018['bq2'] == work_company_type) & (train_2018['bq8_2'] == social_reputation_2)]['bq12_4'].value_counts().index[0]
#   #   elif len(train_2018[(train_2018['bq12_4'].notnull()) & (train_2018['bq1'] == industry_type) & (train_2018['bq2'] == work_company_type) & (train_2018['bq11'] == working_conditions)]) >= 1:
#   #       # '산업 유형', '근무업체 유형', '근무조건'
#   #       train_2018.loc[index, 'bq12_4'] = train_2018[(train_2018['bq12_4'].notnull()) & (train_2018['bq1'] == industry_type) & (train_2018['bq2'] == work_company_type) & (train_2018['bq11'] == working_conditions)]['bq12_4'].value_counts().index[0]
#   #   elif len(train_2018[(train_2018['bq12_4'].notnull()) & (train_2018['bq1'] == industry_type) & (train_2018['bq2'] == work_company_type)]) >= 1:
#   #       #  '산업 유형', '근무업체 유형'
#   #       train_2018.loc[index, 'bq12_4'] = train_2018[(train_2018['bq12_4'].notnull()) & (train_2018['bq1'] == industry_type) & (train_2018['bq2'] == work_company_type)]['bq12_4'].value_counts().index[0]
#   #   elif len(train_2018[(train_2018['bq12_4'].notnull()) & (train_2018['bq1'] == industry_type) & (train_2018['bq8_2'] == social_reputation_2)]) >= 1:
#   #       # '산업 유형', '사회적 평판'
#   #       train_2018.loc[index, 'bq12_4'] = train_2018[(train_2018['bq12_4'].notnull()) & (train_2018['bq1'] == industry_type) & (train_2018['bq8_2'] == social_reputation_2)]['bq12_4'].value_counts().index[0]
#   #   else:
#   #       # 학습 데이터에서는 위의 가정에 모두 속해있다.
#   #       pass
#   # test_indexs = test_2018[(test_2018['bq12_4'].isnull())].index
#   # for index in test_2018[(test_2018['bq12_4'].isnull())].index:
#   #   # 산업 유형
#   #   industry_type = test_2018.loc[index, 'bq1']
#   #   # 근무업체 유형
#   #   work_company_type = test_2018.loc[index, 'bq2']
#   #   # 교육수준
#   #   education_level = test_2018.loc[index, 'bq7']
#   #   # 사회적 평판_1
#   #   social_reputation_1 = test_2018.loc[index, 'bq8_1']
#   #   # 사회적 평판_2
#   #   social_reputation_2 = test_2018.loc[index, 'bq8_2']
#   #   # 직업 안정성
#   #   job_security = test_2018.loc[index, 'bq9']
#   #   # 근무조건
#   #   working_conditions = test_2018.loc[index, 'bq11']
#   #   # 발전가능성
#   #   advancement  = test_2018.loc[index, 'bq10']
    
#   #   # 동료에 대한 좋고 나쁨은 위의 상사와 비슷하다고 생각한다.
#   #   # 따라서 동일한 가정을 통해서 추측하도록 하겠다. 
#   #       # bq1, bq2, bq8_2, bq11
#   #   if len(test_2018[(test_2018['bq12_4'].notnull()) & (test_2018['bq1'] == industry_type) & (test_2018['bq2'] == work_company_type) & (test_2018['bq8_2'] == social_reputation_2) & (test_2018['bq11'] == working_conditions)]) >= 1:
#   #       # '산업 유형', '근무업체 유형', '사회적 평판', '근무조건'
#   #       test_2018.loc[index, 'bq12_4'] = test_2018[(test_2018['bq12_4'].notnull()) & (test_2018['bq1'] == industry_type) & (test_2018['bq2'] == work_company_type) & (test_2018['bq8_2'] == social_reputation_2) & (test_2018['bq11'] == working_conditions)]['bq12_4'].value_counts().index[0]
#   #   elif len(test_2018[(test_2018['bq12_4'].notnull()) & (test_2018['bq1'] == industry_type) & (test_2018['bq2'] == work_company_type) & (test_2018['bq8_2'] == social_reputation_2)]) >= 1:
#   #       # '산업 유형', '근무업체 유형', '사회적 평판'
#   #       test_2018.loc[index, 'bq12_4'] = test_2018[(test_2018['bq12_4'].notnull()) & (test_2018['bq1'] == industry_type) & (test_2018['bq2'] == work_company_type) & (test_2018['bq8_2'] == social_reputation_2)]['bq12_4'].value_counts().index[0]
#   #   elif len(test_2018[(test_2018['bq12_4'].notnull()) & (test_2018['bq1'] == industry_type) & (test_2018['bq2'] == work_company_type) & (test_2018['bq11'] == working_conditions)]) >= 1:
#   #       # '산업 유형', '근무업체 유형', '근무조건'
#   #       test_2018.loc[index, 'bq12_4'] = test_2018[(test_2018['bq12_4'].notnull()) & (test_2018['bq1'] == industry_type) & (test_2018['bq2'] == work_company_type) & (test_2018['bq11'] == working_conditions)]['bq12_4'].value_counts().index[0]
#   #   elif len(test_2018[(test_2018['bq12_4'].notnull()) & (test_2018['bq1'] == industry_type) & (test_2018['bq2'] == work_company_type)]) >= 1:
#   #       #  '산업 유형', '근무업체 유형'
#   #       test_2018.loc[index, 'bq12_4'] = test_2018[(test_2018['bq12_4'].notnull()) & (test_2018['bq1'] == industry_type) & (test_2018['bq2'] == work_company_type)]['bq12_4'].value_counts().index[0]
#   #   elif len(test_2018[(test_2018['bq12_4'].notnull()) & (test_2018['bq1'] == industry_type) & (test_2018['bq8_2'] == social_reputation_2)]) >= 1:
#   #       # '산업 유형', '사회적 평판'
#   #       test_2018.loc[index, 'bq12_4'] = test_2018[(test_2018['bq12_4'].notnull()) & (test_2018['bq1'] == industry_type) & (test_2018['bq8_2'] == social_reputation_2)]['bq12_4'].value_counts().index[0]
#   #   else:
#   #       # 테스트 데이터에서는 위의 가정에 모두 속해있다.
#   #       pass
    
#   train_2018['bq223'].replace('3.0', '3', inplace = True)
#   train_2018['bq223'].replace(3.0, '3', inplace = True)

#   test_2018['bq223'].replace('3.0', '3', inplace = True)
#   test_2018['bq223'].replace(3.0, '3', inplace = True)

#   train_2018.loc[(train_2018['bq223'] != '3'), 'bq223'] = np.nan
#   test_2018.loc[(test_2018['bq223'] != '3'), 'bq223'] = np.nan

#   train_2018.loc[(train_2018['bq223'] == '3'), 'bq221'] = np.nan
#   train_2018.loc[(train_2018['bq223'] == '3'), 'bq222'] = np.nan

#   test_2018.loc[(test_2018['bq223'] == '3'), 'bq221'] = np.nan
#   test_2018.loc[(test_2018['bq223'] == '3'), 'bq222'] = np.nan

#   train_2018['bq221'].replace('2', np.nan, inplace = True)
#   train_2018['bq221'].replace('3', np.nan, inplace = True)
#   test_2018['bq221'].replace('2', np.nan, inplace = True)
#   test_2018['bq221'].replace('3', np.nan, inplace = True)

#   train_2018['bq222'].replace('0', np.nan, inplace = True)
#   test_2018['bq222'].replace('0', np.nan, inplace = True)
#   test_2018['bq222'].replace('1', np.nan, inplace = True)
#   test_2018['bq222'].replace('3', np.nan, inplace = True)

#   for col in ['bq221', 'bq222', 'bq223']:
#     train_2018[col].replace(np.nan, '없음', inplace = True)
#     test_2018[col].replace(np.nan, '없음', inplace = True)
#   train_2018.loc[(train_2018['bq231'] != '1'), 'bq231'] = np.nan
#   test_2018.loc[(test_2018['bq231'] != '1'), 'bq231'] = np.nan

#   train_2018.loc[(train_2018['bq232'] != 2), 'bq232'] = np.nan
#   test_2018.loc[(test_2018['bq232'] != 2), 'bq232'] = np.nan

#   train_2018.loc[(train_2018['bq233'] != 3), 'bq233'] = np.nan
#   test_2018.loc[(test_2018['bq233'] != 3), 'bq233'] = np.nan

#   train_2018['bq234'].replace('4.0', '4', inplace = True)
#   test_2018['bq234'].replace('4.0', '4', inplace = True)
#   train_2018.loc[(train_2018['bq234'] != '4'), 'bq234'] = np.nan
#   test_2018.loc[(test_2018['bq234'] != '4'), 'bq234'] = np.nan

#   train_2018.loc[(train_2018['bq235'] != '5'), 'bq235'] = np.nan
#   test_2018.loc[(test_2018['bq235'] != '5'), 'bq235'] = np.nan

#   train_2018 = train_2018.drop(columns = ['bq231', 'bq232', 'bq233', 'bq234', 'bq235'])
#   test_2018 = test_2018.drop(columns = ['bq231', 'bq232', 'bq233', 'bq234', 'bq235'])

#   train_2018['bq25'].replace(1, '1', inplace = True)
#   train_2018['bq25'].replace(2, '2', inplace = True)
#   train_2018['bq25'].replace(3, '3', inplace = True)

#   test_2018['bq25'].replace(1.0, '1', inplace = True)
#   test_2018['bq25'].replace(2.0, '2', inplace = True)
#   test_2018['bq25'].replace(3.0, '3', inplace = True)

#   train_2018.loc[(train_2018['bq25'].isnull()), 'bq25'] = '1'
#   test_2018.loc[(test_2018['bq25'].isnull()), 'bq25'] = '1'

#   train_2018.loc[(train_2018['bq25'] == '1') & (train_2018['bq25_1'].isnull()), 'bq25_1'] = '2'
#   test_2018.loc[(test_2018['bq25'] == '1') & (test_2018['bq25_1'].isnull()), 'bq25_1'] = '2'
#   train_2018['bq25_1'].replace('3', '2', inplace = True)
#   test_2018['bq25_1'].replace('3', '2', inplace = True)
#   train_2018['bq25_1'].replace(np.nan, '없음', inplace = True)
#   test_2018['bq25_1'].replace(np.nan, '없음', inplace = True)

#   train_2018.replace('1.0', '1', inplace = True)
#   train_2018.replace(1.0, '1', inplace = True)

#   test_2018.replace('1.0', '1', inplace = True)
#   test_2018.replace(1.0, '1', inplace = True)

#   train_2018.replace('2.0', '2', inplace = True)
#   train_2018.replace(2.0, '2', inplace = True)

#   test_2018.replace('2.0', '2', inplace = True)
#   test_2018.replace(2.0, '2', inplace = True)

#   train_2018.replace('3.0', '3', inplace = True)
#   train_2018.replace(3.0, '3', inplace = True)

#   test_2018.replace('3.0', '3', inplace = True)
#   test_2018.replace(3.0, '3', inplace = True)

#   train_2018.loc[(train_2018['bq26_1'].isnull()), 'bq26_1'] = '1'
#   test_2018.loc[(test_2018['bq26_1'].isnull()), 'bq26_1'] = '1'

#   train_2018.loc[(train_2018['bq28_1'].isnull()), 'bq28_1'] = '없음'
#   test_2018.loc[(test_2018['bq28_1'].isnull()), 'bq28_1'] = '없음'

#   train_2018.loc[(train_2018['bq29'].isnull()), 'bq29'] = '없음'
#   test_2018.loc[(test_2018['bq29'].isnull()), 'bq29'] = '없음'

#   train_2018.loc[(train_2018['bq31'].isnull()), 'bq31'] = '없음'
#   test_2018.loc[(test_2018['bq31'].isnull()), 'bq31'] = '없음'

#   train_2018.loc[(train_2018['bq32'].isnull()), 'bq32'] = '없음'
#   test_2018.loc[(test_2018['bq32'].isnull()), 'bq32'] = '없음'

#   train_2018.loc[(train_2018['bq33'].isnull()), 'bq33'] = '없음'
#   test_2018.loc[(test_2018['bq33'].isnull()), 'bq33'] = '없음'

#   train_2018.loc[(train_2018['bq36'] == '디자이너'), 'bq36'] = train_2018.loc[(train_2018['bq36'].notnull() & (train_2018['bq36'] != '디자이너'))]['bq36'].astype('int').quantile(q = 0.5)
#   train_2018.loc[(train_2018['bq36'].isnull()), 'bq36'] = train_2018.loc[(train_2018['bq36'].notnull() & (train_2018['bq36'] != '디자이너'))]['bq36'].astype('int').quantile(q = 0.5)
#   train_2018.loc[(train_2018['bq36'] == '없음'), 'bq36'] = -1
#   train_2018['bq36'].replace('2', '20', inplace = True)
#   train_2018['bq36'].replace('3', '30', inplace = True)
#   train_2018['bq36'].replace('4', '40', inplace = True)
#   train_2018['bq36'].replace(5.0, '50', inplace = True)

#   train_2018.loc[(train_2018['bq37'] == '1'), 'bq37_1'] = '없음'
#   test_2018.loc[(test_2018['bq37'] == '1'), 'bq37_1'] = '없음'

#   haha = ['1', '2', '3', '4', '5', '6']
#   train_indexs = []
#   for index in train_2018.index:
#     value = train_2018.loc[index, 'bq37']
    
#     if value not in haha:
#       train_indexs.append(index)
#       if value != '공고졸업':
#         train_2018.loc[index, 'bq37'] = '4'
#         train_2018.loc[index, 'bq37_1'] = value
#       else:
#         train_2018.loc[index, 'bq37'] = '2'
#         train_2018.loc[index, 'bq37_1'] = value
              
#   for index in test_2018.index:
#     value = test_2018.loc[index, 'bq37']
    
#     if value not in haha:
#       if value != '공고졸업':
#         test_2018.loc[index, 'bq37'] = '4'
#         test_2018.loc[index, 'bq37_1'] = value
#       else:
#         test_2018.loc[index, 'bq37'] = '2'
#         test_2018.loc[index, 'bq37_1'] = value

#   #bq38_1
#   train_2018.loc[train_2018['bq38_1'].isnull()] = '없음'
#   test_2018.loc[test_2018['bq38_1'].isnull()] = '없음'

  
#   train_2018.loc[train_2018['bq40'] == '없음'] = np.nan
#   test_2018.loc[test_2018['bq40'] == '없음']= np.nan

#   train_2018.loc[train_2018['bq40'] == '지식재산학'] = np.nan

#   # #bq39_1
#   # train_2018.loc[(train_2018['bq39_1'] == 2) & (train_2018['bq40'].isnull())] = '3'
#   # test_2018.loc[(test_2018['bq39_1'] == 2) & (test_2018['bq40'].isnull())] = '3'

#   return (train_2018, test_2018)

# train[1], test[1] = preprocessing_2018(train[1], test[1])
# train[1]['knowcode'].value_counts()

In [ ]:
# def preprocessing_2017(train_2017, test_2017):
#   columns = train_2017.columns[1:-1][:82]

#   # 앞의 결측치 값을 제거하면 뒤의 결측치 값은 제거하면 되기 때문에
#   # 데이터의 인덱스를 2씩 키운다.
#   for index in range(0,82,2):
#       col = columns[index]
      
#       # before가 가번 문항에 해당한다. 
#       # after는 나번 문항에 해당한다. 
#       before = col
#       after = col[:-1] + str((int(col[-1])+1))
      
#       # 일단 before 자체도 결측치 값이면 안된다.
#       # 그리고 after 값이 결측치 값인 것을 확인한다.
#       # after 값이 결측치 값이라는 것은 가번 문항에서 1번 을 선택한 것이기 때문에 0으로 할당해 줘야 한다.
#       train_2017.loc[(train_2017[after].isnull()) & (train_2017[before].notnull()) & (train_2017[before] == (sorted(train_2017[before].unique()))[0]), after] = 0
#       test_2017.loc[(test_2017[after].isnull()) & (test_2017[before].notnull()) & (test_2017[before] == (sorted(test_2017[before].unique()))[0]), after] = 0

#   train_nan_columns = []
#   test_nan_columns = []
#   for index, value, tindex, tvalue in zip(train_2017.isnull().sum().index, train_2017.isnull().sum(), test_2017.isnull().sum().index, test_2017.isnull().sum()):
#       if value != 0 :
#           train_nan_columns.append(index)
#       if tvalue != 0:
#           test_nan_columns.append(tindex)
  
#   #aq11_2
#   train_2017.loc[train_2017['aq11_2'].isnull(), 'aq11_2'] = train_2017.loc[(train_2017['aq11_1'] == 3) & (train_2017['aq11_2'].notnull())]['aq11_2'].astype('int').quantile(q = 0.5)
#   test_2017.loc[test_2017['aq11_2'].isnull(), 'aq11_2'] = test_2017.loc[(test_2017['aq11_1'] == 3) & (test_2017['aq11_2'].notnull())]['aq11_2'].astype('int').quantile(q = 0.5)

#   #aq14_2
#   # 가번 문항에서 3번을 선택한 사람의 중앙 값을 알기 위해서 전체 데이터로 합친다.
#   train_2017.loc[(train_2017['aq14_2'].isnull()) & (train_2017['aq14_1'] == 3), 'aq14_2'] = train_2017.loc[(train_2017['aq14_1'] == 3) & (train_2017['aq14_2'].notnull())]['aq14_2'].astype('int').quantile(q = 0.5)
#   test_2017.loc[(test_2017['aq14_2'].isnull()) & (test_2017['aq14_1'] == 3), 'aq14_2'] = test_2017.loc[(test_2017['aq14_1'] == 3) & (test_2017['aq14_2'].notnull())]['aq14_2'].astype('int').quantile(q = 0.5)
#   # 가번 문항에서 4번을 선택한 사람의 중앙 값을 알기 위해서 전체 데이터로 합친다.
#   train_2017.loc[(train_2017['aq14_2'].isnull()) & (train_2017['aq14_1'] == 4), 'aq14_2'] = train_2017.loc[(train_2017['aq14_1'] == 4) & (train_2017['aq14_2'].notnull())]['aq14_2'].astype('int').quantile(q = 0.5)
#   test_2017.loc[(test_2017['aq14_2'].isnull()) & (test_2017['aq14_1'] == 4), 'aq14_2'] = test_2017.loc[(test_2017['aq14_1'] == 4) & (test_2017['aq14_2'].notnull())]['aq14_2'].astype('int').quantile(q = 0.5)

#   #aq27_2
#   train_2017.loc[(train_2017['aq27_2'].isnull()) & (train_2017['aq27_1'] == 3), 'aq27_2'] = train_2017.loc[(train_2017['aq27_1'] == 3) & (train_2017['aq27_2'].notnull())]['aq27_2'].astype('int').quantile(q = 0.5)
#   test_2017.loc[(test_2017['aq27_2'].isnull()) & (test_2017['aq27_1'] == 3), 'aq27_2'] = test_2017.loc[(test_2017['aq27_1'] == 3) & (test_2017['aq27_2'].notnull())]['aq27_2'].astype('int').quantile(q = 0.5)
#   # 가번 문항에서 4번을 선택한 사람을 처리한다.[
#   train_2017.loc[(train_2017['aq27_2'].isnull()) & (train_2017['aq27_1'] == 4), 'aq27_2'] = train_2017.loc[(train_2017['aq27_1'] == 4) & (train_2017['aq27_2'].notnull())]['aq27_2'].astype('int').quantile(q = 0.5)
#   test_2017.loc[(test_2017['aq27_2'].isnull()) & (test_2017['aq27_1'] == 4), 'aq27_2'] = test_2017.loc[(test_2017['aq27_1'] == 3) & (test_2017['aq27_2'].notnull())]['aq27_2'].astype('int').quantile(q = 0.5)

#   #aq32_2
#   # train_2017.loc[(train_2017['aq32_2'].isnull()) & (train_2017['aq32_1'] == 3)] = train_2017.loc[(train_2017['aq32_1'] == 3) & (train_2017['aq32_2'].notnull())]['aq32_2'].astype('int').quantile(q = 0.5)
#   # test_2017.loc[(test_2017['aq32_2'].isnull()) & (test_2017['aq32_1'] == 3)] = test_2017.loc[(test_2017['aq32_1'] == 3) & (test_2017['aq32_2'].notnull())]['aq32_2'].astype('int').quantile(q = 0.5)

#   #aq41_2
#   train_2017.loc[(train_2017['aq41_2'].isnull()) & (train_2017['aq41_1'] == 3), 'aq41_2'] = train_2017.loc[(train_2017['aq41_1'] == 3) & (train_2017['aq41_2'].notnull())]['aq41_2'].astype('int').quantile(q = 0.5)
#   test_2017.loc[(test_2017['aq41_2'].isnull()) & (test_2017['aq41_1'] == 3), 'aq41_2'] = test_2017.loc[(test_2017['aq41_1'] == 3) & (test_2017['aq41_2'].notnull())]['aq41_2'].astype('int').quantile(q = 0.5)

#   #bq4_1a~bq4_1c
#   train_2017.loc[(train_2017['bq4'] == 2) & (train_2017['bq4_1a'].isnull()), 'bq4_1a'] = '없음'
#   test_2017.loc[(test_2017['bq4'] == 2) & (test_2017['bq4_1a'].isnull()), 'bq4_1a'] = '없음'
#   train_2017.loc[(train_2017['bq4'] == 2) & (train_2017['bq4_1b'].isnull()), 'bq4_1b'] = '없음'
#   train_2017.loc[(train_2017['bq4'] == 2) & (train_2017['bq4_1c'].isnull()), 'bq4_1c'] = '없음'
#   test_2017.loc[(test_2017['bq4'] == 2) & (test_2017['bq4_1b'].isnull()), 'bq4_1b'] = '없음'
#   test_2017.loc[(test_2017['bq4'] == 2) & (test_2017['bq4_1c'].isnull()), 'bq4_1c'] = '없음'
#   for index in train_2017.loc[(train_2017['bq4_1b'].isnull())]['bq4_1a'].index:
#     value = train_2017.iloc[index]['bq4_1a']
#     if len(train_2017[(train_2017['bq4_1a'] == value) & (train_2017['bq4_1b'].notnull()) & (train_2017['bq4_1b'] != '없음') & (train_2017['bq4_1b'] != '없다')]) > 0:
#          train_2017.loc[index, 'bq4_1b'] = train_2017[(train_2017['bq4_1a'] == value) & (train_2017['bq4_1b'].notnull()) & (train_2017['bq4_1b'] != '없음') & (train_2017['bq4_1b'] != '없다')]['bq4_1b'].value_counts().index[0]
#   for index in test_2017.loc[(test_2017['bq4_1b'].isnull())].index:
#     value = test_2017.iloc[index]['bq4_1a']
#     if len(test_2017[(test_2017['bq4_1a'] == value) & (test_2017['bq4_1b'].notnull()) & (test_2017['bq4_1b'] != '없음') & (test_2017['bq4_1b'] != '없다')]) > 0:
#         test_2017.loc[index, 'bq4_1b'] = test_2017[(test_2017['bq4_1a'] == value) & (test_2017['bq4_1b'].notnull()) & (test_2017['bq4_1b'] != '없음') & (test_2017['bq4_1b'] != '없다')]['bq4_1b'].value_counts().index[0]
#   train_2017.loc[(train_2017['bq4_1b'].isnull()), 'bq4_1b'] = '없음'
#   test_2017.loc[(test_2017['bq4_1b'].isnull()), 'bq4_1b'] = '없음'
#   train_2017.loc[(train_2017['bq4_1c'].isnull()), 'bq4_1c'] = '없음'
#   test_2017.loc[(test_2017['bq4_1c'].isnull()), 'bq4_1c'] = '없음'

#   # '없다'라는 값을 '없음'으로 통일해 준다.
#   for col in test_2017:
#       train_2017[col].replace('없다','없음', inplace = True)
#       test_2017[col].replace('없다','없음', inplace = True)
#   #bq5_1~bq5_2
#   train_2017.loc[(train_2017['bq5'] == 2) & (train_2017['bq5_1'].isnull()),'bq5_1'] = 0
#   test_2017.loc[(test_2017['bq5'] == 2) & (test_2017['bq5_1'].isnull()),'bq5_1'] = 0
#   train_2017.loc[(train_2017['bq5'] == 2) & (train_2017['bq5_2'].isnull()),'bq5_2'] = '없음'
#   test_2017.loc[(test_2017['bq5'] == 2) & (test_2017['bq5_2'].isnull()),'bq5_2'] = '없음'
#   train_2017.loc[train_2017['bq5_2'].isnull(), 'bq5_2'] = '없음'
#   test_2017.loc[test_2017['bq5_2'].isnull(), 'bq5_2'] = '없음'

#   # bq12
#   for index in train_2017.loc[(train_2017['bq12_2'].isnull())].index:
#     # 학력 값을 저장함
#     LevelOfEducation = train_2017.iloc[index]['bq7']

#     # 학력 값에 대한 중앙값을 통해서 결측치를 제거한다.
#     train_2017.loc[index, 'bq12_2'] = train_2017[(train_2017['bq7'] == LevelOfEducation) & (train_2017['bq12_2'].notnull())]['bq12_2'].astype('int').quantile(q=0.5).astype('int').astype('str')
    
#   for index in test_2017.loc[(test_2017['bq12_2'].isnull())].index:
#     # 학력 값을 저장함
#     LevelOfEducation = test_2017.iloc[index]['bq7']

#     # 학력 값에 대한 중앙값을 통해서 결측치를 제거한다.
#     test_2017.loc[index, 'bq12_2'] = test_2017[(test_2017['bq7'] == LevelOfEducation) & (test_2017['bq12_2'].notnull())]['bq12_2'].astype('int').quantile(q=0.5).astype('int').astype('str')

#   for index in train_2017.loc[(train_2017['bq12_3'].isnull())].index:
#     # 학력 값을 저장함
#     LevelOfEducation = train_2017.iloc[index]['bq7']

#     # 학력 값에 대한 중앙값을 통해서 결측치를 제거한다.
#     train_2017.loc[index, 'bq12_3'] = train_2017[(train_2017['bq7'] == LevelOfEducation) & (train_2017['bq12_3'].notnull())]['bq12_3'].astype('int').quantile(q=0.5).astype('int').astype('str')
    
#   for index in test_2017.loc[(test_2017['bq12_3'].isnull())].index:
#     # 학력 값을 저장함
#     LevelOfEducation = test_2017.iloc[index]['bq7']

#     # 학력 값에 대한 중앙값을 통해서 결측치를 제거한다.
#     test_2017.loc[index, 'bq12_3'] = test_2017[(test_2017['bq7'] == LevelOfEducation) & (test_2017['bq12_3'].notnull())]['bq12_3'].astype('int').quantile(q=0.5).astype('int').astype('str')
#   for index in train_2017.loc[(train_2017['bq12_4'].isnull())].index:
#     # 학력 값을 저장함
#     LevelOfEducation = train_2017.iloc[index]['bq7']

#     # 학력 값에 대한 중앙값을 통해서 결측치를 제거한다.
#     train_2017.loc[index, 'bq12_4'] = train_2017[(train_2017['bq7'] == LevelOfEducation) & (train_2017['bq12_4'].notnull())]['bq12_4'].astype('int').quantile(q=0.5).astype('int').astype('str')

#   for index in test_2017.loc[(test_2017['bq12_4'].isnull())].index:
#     # 학력 값을 저장함
#     LevelOfEducation = test_2017.iloc[index]['bq7']

#     # 학력 값에 대한 중앙값을 통해서 결측치를 제거한다.
#     test_2017.loc[index, 'bq12_4'] = test_2017[(train_2017['bq7'] == LevelOfEducation) & (test_2017['bq12_4'].notnull())]['bq12_4'].astype('int').quantile(q=0.5).astype('int').astype('str')

#   #bq19
#   train_2017.loc[train_2017['bq19_1'].isnull(), 'bq19_1'] = '없음'
#   test_2017.loc[test_2017['bq19_1'].isnull(), 'bq19_1'] = '없음'
#   #bq30
#   train_2017.loc[train_2017['bq30'].isnull(), 'bq30'] = '없음'
#   test_2017.loc[test_2017['bq30'].isnull(), 'bq30'] = '없음'
#   #bq31
#   # for index in train_2017[(train_2017['bq31'].isnull())].index:
#   #   # 산업 유형
#   #   industry_type = train_2017.iloc[index]['bq1']
#   #   # 유사직업명
#   #   a_similar_job = train_2017.iloc[index]['bq30']
#   #   # 직전직업
#   #   before_job = train_2017.iloc[index]['bq32']
#   #   # 전직 가능직업
#   #   can_change_job = train_2017.iloc[index]['bq33']
    

#   #   if len(train_2017[(train_2017['bq31'].notnull())&(train_2017['bq1'] == industry_type) & (train_2017['bq30'] == a_similar_job) & (train_2017['bq32'] == before_job) & (train_2017['bq33'] == can_change_job)]):
#   #       # 산업 유형 + 유사 직업명 + 직전 직업 + 전직 가능 직업
#   #       train_2017.loc[index, 'bq31'] = train_2017[(train_2017['bq31'].notnull())&(train_2017['bq1'] == industry_type) & (train_2017['bq30'] == a_similar_job) & (train_2017['bq32'] == before_job) & (train_2017['bq33'] == can_change_job)]['bq31'].value_counts().index[0]
#   #   elif len(train_2017[(train_2017['bq31'].notnull())&(train_2017['bq1'] == industry_type) & (train_2017['bq30'] == a_similar_job) & (train_2017['bq32'] == before_job)]):
#   #       # 산업 유형 + 유사 직업명 + 직전 직업
#   #       train_2017.loc[index, 'bq31'] = train_2017[(train_2017['bq31'].notnull())&(train_2017['bq1'] == industry_type) & (train_2017['bq30'] == a_similar_job) & (train_2017['bq32'] == before_job)]['bq31'].value_counts().index[0]
#   #   elif len(train_2017[(train_2017['bq31'].notnull())& (train_2017['bq30'] == a_similar_job) & (train_2017['bq32'] == before_job) & (train_2017['bq33'] == can_change_job)]):
#   #       #  유사 직업명 + 직전 직업 + 전직 가능 직업
#   #       train_2017.loc[index, 'bq31'] = train_2017[(train_2017['bq31'].notnull())& (train_2017['bq30'] == a_similar_job) & (train_2017['bq32'] == before_job) & (train_2017['bq33'] == can_change_job)]['bq31'].value_counts().index[0]
#   #   elif len(train_2017[(train_2017['bq31'].notnull())&(train_2017['bq1'] == industry_type) & (train_2017['bq30'] == a_similar_job)]):
#   #       # 산업 유형 + 유사 직업명
#   #       train_2017.loc[index, 'bq31'] = train_2017[(train_2017['bq31'].notnull())&(train_2017['bq1'] == industry_type) & (train_2017['bq30'] == a_similar_job)]['bq31'].value_counts().index[0]
#   #   elif len(train_2017[(train_2017['bq31'].notnull())&(train_2017['bq1'] == industry_type) & (train_2017['bq30'] == a_similar_job) & (train_2017['bq32'] == before_job)]):
#   #       # 산업 유형 + 직전 직업
#   #       train_2017.loc[index, 'bq31'] = train_2017[(train_2017['bq31'].notnull())&(train_2017['bq1'] == industry_type) & (train_2017['bq30'] == a_similar_job) & (train_2017['bq32'] == before_job)]['bq31'].value_counts().index[0]
#   #   elif len(train_2017[(train_2017['bq31'].notnull())& (train_2017['bq32'] == before_job) & (train_2017['bq33'] == can_change_job)]):
#   #       # 직전 직업 + 전직 가능직업
#   #       train_2017.loc[index, 'bq31'] = train_2017[(train_2017['bq31'].notnull())& (train_2017['bq32'] == before_job) & (train_2017['bq33'] == can_change_job)]['bq31'].value_counts().index[0]
#   #   elif len(train_2017[(train_2017['bq31'].notnull())& (train_2017['bq32'] == before_job)]):
#   #       # 직전 직업
#   #       train_2017.loc[index, 'bq31'] = train_2017[(train_2017['bq31'].notnull()) & (train_2017['bq32'] == before_job)]['bq31'].value_counts().index[0]
#   #   elif len(train_2017[(train_2017['bq31'].notnull()) & (train_2017['bq33'] == can_change_job)]):
#   #       # 전직 가능직업
#   #       train_2017.loc[index, 'bq31'] = train_2017[(train_2017['bq31'].notnull()) & (train_2017['bq33'] == can_change_job)]['bq31'].value_counts().index[0]
#   #   elif len(train_2017[(train_2017['bq31'].notnull())& (train_2017['bq30'] == a_similar_job)]):
#   #       # 유사 직업명
#   #       train_2017.loc[index, 'bq31'] = train_2017[(train_2017['bq31'].notnull())& (train_2017['bq30'] == a_similar_job)]['bq31'].value_counts().index[0]
#   #   else:
#   #       # 이정도면 없음으로 할게요...
#   #       train_2017.loc[index, 'bq31'] = '없음'

#   # for index in test_2017[(test_2017['bq31'].isnull())].index:
#   #   # 산업 유형
#   #   industry_type = test_2017.iloc[index]['bq1']
#   #   # 유사직업명
#   #   a_similar_job = test_2017.iloc[index]['bq30']
#   #   # 직전직업
#   #   before_job = test_2017.iloc[index]['bq32']
#   #   # 전직 가능직업
#   #   can_change_job = test_2017.iloc[index]['bq33']

#   #   if len(test_2017[(test_2017['bq31'].notnull()) & (test_2017['bq1'] == industry_type) & (
#   #           test_2017['bq30'] == a_similar_job) & (test_2017['bq32'] == before_job) & (
#   #                             test_2017['bq33'] == can_change_job)]):
#   #       # 산업 유형 + 유사 직업명 + 직전 직업 + 전직 가능 직업
#   #       test_2017.loc[index, 'bq31'] = test_2017[
#   #           (test_2017['bq31'].notnull()) & (test_2017['bq1'] == industry_type) & (
#   #                       test_2017['bq30'] == a_similar_job) & (test_2017['bq32'] == before_job) & (
#   #                       test_2017['bq33'] == can_change_job)]['bq31'].value_counts().index[0]
#   #   elif len(test_2017[(test_2017['bq31'].notnull()) & (test_2017['bq1'] == industry_type) & (
#   #           test_2017['bq30'] == a_similar_job) & (test_2017['bq32'] == before_job)]):
#   #       # 산업 유형 + 유사 직업명 + 직전 직업
#   #       test_2017.loc[index, 'bq31'] = test_2017[
#   #           (test_2017['bq31'].notnull()) & (test_2017['bq1'] == industry_type) & (
#   #                       test_2017['bq30'] == a_similar_job) & (test_2017['bq32'] == before_job)][
#   #           'bq31'].value_counts().index[0]
#   #   elif len(test_2017[(test_2017['bq31'].notnull()) & (test_2017['bq30'] == a_similar_job) & (
#   #           test_2017['bq32'] == before_job) & (test_2017['bq33'] == can_change_job)]):
#   #       #  유사 직업명 + 직전 직업 + 전직 가능 직업
#   #       test_2017.loc[index, 'bq31'] = test_2017[
#   #           (test_2017['bq31'].notnull()) & (test_2017['bq30'] == a_similar_job) & (
#   #                       test_2017['bq32'] == before_job) & (test_2017['bq33'] == can_change_job)][
#   #           'bq31'].value_counts().index[0]
#   #   elif len(test_2017[(test_2017['bq31'].notnull()) & (test_2017['bq1'] == industry_type) & (
#   #           test_2017['bq30'] == a_similar_job)]):
#   #       # 산업 유형 + 유사 직업명
#   #       test_2017.loc[index, 'bq31'] = test_2017[
#   #           (test_2017['bq31'].notnull()) & (test_2017['bq1'] == industry_type) & (
#   #                       test_2017['bq30'] == a_similar_job)]['bq31'].value_counts().index[0]
#   #   elif len(test_2017[(test_2017['bq31'].notnull()) & (test_2017['bq1'] == industry_type) & (
#   #           test_2017['bq30'] == a_similar_job) & (test_2017['bq32'] == before_job)]):
#   #       # 산업 유형 + 직전 직업
#   #       test_2017.loc[index, 'bq31'] = test_2017[
#   #           (test_2017['bq31'].notnull()) & (test_2017['bq1'] == industry_type) & (
#   #                       test_2017['bq30'] == a_similar_job) & (test_2017['bq32'] == before_job)].value_counts().index[0]
#   #   elif len(test_2017[(test_2017['bq31'].notnull()) & (test_2017['bq32'] == before_job) & (
#   #           test_2017['bq33'] == can_change_job)]):
#   #       # 직전 직업 + 전직 가능직업
#   #       test_2017.loc[index, 'bq31'] = test_2017[
#   #           (test_2017['bq31'].notnull()) & (test_2017['bq32'] == before_job) & (
#   #                       test_2017['bq33'] == can_change_job)].value_counts().index[0]
#   #   elif len(test_2017[(test_2017['bq31'].notnull()) & (test_2017['bq32'] == before_job)]):
#   #       # 직전 직업
#   #       test_2017.loc[index, 'bq31'] = \
#   #       test_2017[(test_2017['bq31'].notnull()) & (test_2017['bq32'] == before_job)].value_counts().index[0]
#   #   elif len(test_2017[(test_2017['bq31'].notnull()) & (test_2017['bq33'] == can_change_job)]):
#   #       # 전직 가능직업
#   #       test_2017.loc[index, 'bq31'] = \
#   #       test_2017[(test_2017['bq31'].notnull()) & (test_2017['bq33'] == can_change_job)].value_counts().index[0]
#   #   elif len(test_2017[(test_2017['bq31'].notnull()) & (test_2017['bq30'] == a_similar_job)]):
#   #       # 유사 직업명
#   #       test_2017.loc[index, 'bq31'] = \
#   #       test_2017[(test_2017['bq31'].notnull()) & (test_2017['bq30'] == a_similar_job)].value_counts().index[0]
#   #   else:
#   #       # 이정도면 없음으로 할게요...
#   #       test_2017.loc[index, 'bq31'] = '없음'

#   #bq32
#   train_2017.loc[train_2017['bq32'].isnull(), 'bq32'] = '모름' 
#   test_2017.loc[test_2017['bq32'].isnull(), 'bq32'] = '모름'

#   #bq34
#   train_2017.loc[(train_2017['bq34'].isnull()), 'bq34'] = '모름'
#   test_2017.loc[(test_2017['bq34'].isnull()), 'bq34'] = '모름'

#   return (train_2017, test_2017)

# train[0], test[0] = preprocessing_2017(train[0], test[0])

In [ ]:
# def preprocessing_2019(train_2019, test_2019):
#   train_2019.loc[(train_2019['bq4'] == 2) & (train_2019['bq4_1a'].isnull()), 'bq4_1a'] = '없음'
#   test_2019.loc[(test_2019['bq4'] == 2) & (test_2019['bq4_1a'].isnull()), 'bq4_1a'] = '없음'
#   train_2019.loc[(train_2019['bq4'] == 2) & (train_2019['bq4_1b'].isnull()), 'bq4_1b'] = '없음'
#   train_2019.loc[(train_2019['bq4'] == 2) & (train_2019['bq4_1c'].isnull()), 'bq4_1c'] = '없음'
#   test_2019.loc[(test_2019['bq4'] == 2) & (test_2019['bq4_1b'].isnull()), 'bq4_1b'] = '없음'
#   test_2019.loc[(test_2019['bq4'] == 2) & (test_2019['bq4_1c'].isnull()), 'bq4_1c'] = '없음'
#   train_2019.loc[(train_2019['bq4'] == 1) & (train_2019['bq4_1a'].isnull()), 'bq4_1a'] = '정보처리기사'
#   test_2019.loc[(test_2019['bq4'] == 1) & (test_2019['bq4_1a'].isnull()), 'bq4_1a'] = '정보처리기사'
#   train_2019.loc[(train_2019['bq4_1b'].isnull()), 'bq4_1b'] = '없음'
#   train_2019.loc[(train_2019['bq4_1c'].isnull()), 'bq4_1c'] = '없음'
#   test_2019.loc[(test_2019['bq4_1b'].isnull()), 'bq4_1b'] = '없음'
#   test_2019.loc[(test_2019['bq4_1c'].isnull()), 'bq4_1c'] = '없음'
#   train_2019.loc[(train_2019['bq5'] == 2) & (train_2019['bq5_1'].isnull()),'bq5_1'] = 0
#   test_2019.loc[(test_2019['bq5'] == 2) & (test_2019['bq5_1'].isnull()),'bq5_1'] = 0
#   train_2019.loc[(train_2019['bq5'] == 2) & (train_2019['bq5_2'].isnull()),'bq5_2'] = '없음'
#   test_2019.loc[(test_2019['bq5'] == 2) & (test_2019['bq5_2'].isnull()),'bq5_2'] = '없음'
#   train_2019.loc[train_2019['bq5_2'].isnull(), 'bq5_2'] = '없음'
#   test_2019.loc[test_2019['bq5_2'].isnull(), 'bq5_2'] = '없음'
#   train_2019 = train_2019.drop(columns = ['bq18_10', 'bq20_1', 'bq22', 'bq23', 'bq24'])
#   test_2019 = test_2019.drop(columns = ['bq18_10', 'bq20_1', 'bq22', 'bq23', 'bq24'])
#   train_2019.loc[train_2019['bq27_1'].isnull(), 'bq27_1'] = '없음'
#   test_2019.loc[test_2019['bq27_1'].isnull(), 'bq27_1'] = '없음'
#   train_2019['bq28_1'].replace('없음', np.nan, inplace = True)
#   test_2019['bq28_1'].replace('없음', np.nan, inplace = True)
#   train_2019['bq28_2'].replace('없음', np.nan, inplace = True)
#   test_2019['bq28_2'].replace('없음', np.nan, inplace = True)
#   train_2019['bq28_1'].replace(np.nan, 0,inplace = True)
#   test_2019['bq28_1'].replace(np.nan, 0,inplace = True)
#   train_2019['bq28_2'].replace(np.nan,0, inplace = True)
#   test_2019['bq28_2'].replace(np.nan, 0,inplace = True)
#   train_2019['bq29'].replace(np.nan,0, inplace = True)
#   test_2019['bq29'].replace(np.nan, 0,inplace = True)

#   bq1_list = train_2019.bq1.unique()
#   for bq1 in bq1_list:
#     train_2019.loc[lambda x : (x.bq1 == bq1) & (x.bq31_1.isnull()), 'bq31_1'] = round(train_2019.loc[lambda x: (x.bq1 == bq1) & (x.bq31_1.notnull())].bq31_1.apply(lambda x: int(x)).mean())
#     test_2019.loc[lambda x : (x.bq1 == bq1) & (x.bq31_1.isnull()), 'bq31_1'] = round(test_2019.loc[lambda x: (x.bq1 == bq1) & (x.bq31_1.notnull())].bq31_1.apply(lambda x: int(x)).mean())
    
#     train_2019.loc[lambda x : (x.bq1 == bq1) & (x.bq31_2.isnull()), 'bq31_2'] = round(train_2019.loc[lambda x: (x.bq1 == bq1) & (x.bq31_2.notnull())].bq31_2.apply(lambda x: int(x)).mean())
#     test_2019.loc[lambda x : (x.bq1 == bq1) & (x.bq31_2.isnull()), 'bq31_2'] = round(test_2019.loc[lambda x: (x.bq1 == bq1) & (x.bq31_2.notnull())].bq31_2.apply(lambda x: int(x)).mean())
    
#     try:
#       train_2019.loc[lambda x : (x.bq1 == bq1) & (x.bq31_3.isnull()), 'bq31_3'] = round(train_2019.loc[lambda x: (x.bq1 == bq1) & (x.bq31_3.notnull())].bq31_3.apply(lambda x: int(x)).mean())
#       test_2019.loc[lambda x : (x.bq1 == bq1) & (x.bq31_3.isnull()), 'bq31_3'] = round(test_2019.loc[lambda x: (x.bq1 == bq1) & (x.bq31_3.notnull())].bq31_3.apply(lambda x: int(x)).mean())
#     except:
#       train_2019.loc[lambda x : (x.bq1 == bq1) & (x.bq31_3.isnull()), 'bq31_3'] = round(train_2019.loc[lambda x: (x.bq1 == bq1) & (x.bq31_2.notnull())].bq31_2.apply(lambda x: int(x)).mean())
#       test_2019.loc[lambda x : (x.bq1 == bq1) & (x.bq31_3.isnull()), 'bq31_3'] = round(test_2019.loc[lambda x: (x.bq1 == bq1) & (x.bq31_2.notnull())].bq31_2.apply(lambda x: int(x)).mean())

#   from sklearn.impute import SimpleImputer
#   train_columns = train_2019.columns
#   test_columns = test_2019.columns
#   train_target = train_2019.iloc[:,-1]
#   imp = SimpleImputer(strategy='most_frequent')
#   train_2019_no_target = pd.DataFrame(imp.fit_transform(train_2019.iloc[:,:-1]))
#   test_2019 = pd.DataFrame(imp.transform(test_2019))
#   train_2019 = pd.concat([train_2019_no_target, train_target], axis=1)

#   train_2019.columns = train_columns
#   test_2019.columns = test_columns

#   return (train_2019, test_2019)

# train[2], test[2] = preprocessing_2019(train[2], test[2])

In [ ]:
# def preprocessing_2020(train_2020, test_2020):
#   train_2020.loc[lambda x : (x['saq'+str(40)+'_2'].notnull()) & (x['saq'+str(40)+'_1'] == 1), 'saq40_2'] = np.nan
#   for i in range(1, 45):
#     train_2020.loc[lambda x : (x['saq'+str(i)+'_2'].isnull()) & (x['saq'+str(i)+'_1'] == 1), 'saq'+str(i)+'_2'] = 0
#   test_2020.loc[lambda x : (x['saq'+str(40)+'_2'].isnull()) & (x['saq'+str(40)+'_1'] != 1), 'saq40_2'] = np.nan
#   for i in range(1, 45):
#     test_2020.loc[lambda x : (x['saq'+str(i)+'_2'].isnull()) & (x['saq'+str(i)+'_1'] == 1), 'saq'+str(i)+'_2'] = 0
#   train_2020.loc[(train_2020['bq4'] == 2) & (train_2020['bq4_1a'].isnull()), 'bq4_1a'] = '없음'
#   test_2020.loc[(test_2020['bq4'] == 2) & (test_2020['bq4_1a'].isnull()), 'bq4_1a'] = '없음'
#   train_2020.loc[(train_2020['bq4'] == 2) & (train_2020['bq4_1b'].isnull()), 'bq4_1b'] = '없음'
#   train_2020.loc[(train_2020['bq4'] == 2) & (train_2020['bq4_1c'].isnull()), 'bq4_1c'] = '없음'
#   test_2020.loc[(test_2020['bq4'] == 2) & (test_2020['bq4_1b'].isnull()), 'bq4_1b'] = '없음'
#   test_2020.loc[(test_2020['bq4'] == 2) & (test_2020['bq4_1c'].isnull()), 'bq4_1c'] = '없음'

#   train_2020.loc[(train_2020['bq4_1b'].isnull()), 'bq4_1b'] = '없음'
#   train_2020.loc[(train_2020['bq4_1c'].isnull()), 'bq4_1c'] = '없음'
#   test_2020.loc[(test_2020['bq4_1b'].isnull()), 'bq4_1b'] = '없음'
#   test_2020.loc[(test_2020['bq4_1c'].isnull()), 'bq4_1c'] = '없음'
  
#   for col in test_2020:
#     train_2020[col].replace('없다','없음', inplace = True)
#     test_2020[col].replace('없다','없음', inplace = True)
#   train_2020.loc[(train_2020['bq5'] == 2) & (train_2020['bq5_1'].isnull()),'bq5_1'] = 0
#   test_2020.loc[(test_2020['bq5'] == 2) & (test_2020['bq5_1'].isnull()),'bq5_1'] = 0
#   train_2020.loc[(train_2020['bq5'] == 2) & (train_2020['bq5_2'].isnull()),'bq5_2'] = '없음'
#   test_2020.loc[(test_2020['bq5'] == 2) & (test_2020['bq5_2'].isnull()),'bq5_2'] = '없음'

#   train_2020 = train_2020.drop(columns = ['bq18_10'])
#   test_2020 = test_2020.drop(columns = ['bq18_10'])

#   train_2020.loc[lambda x : x.bq26_1.isnull(), 'bq26_1'] = '없음'
#   test_2020.loc[lambda x : x.bq26_1.isnull(), 'bq26_1'] = '없음'

#   train_2020.loc[lambda x : (x.bq28.isnull()) & (x.bq27_1 == 1), 'bq28'] = 2
#   test_2020.loc[lambda x : (x.bq28.isnull()) & (x.bq27_1 == 1), 'bq28'] = 2
#   train_2020.loc[lambda x : (x.bq28.isnull()) & (x.bq27_1 == 2), 'bq28'] = 1
#   test_2020.loc[lambda x : (x.bq28.isnull()) & (x.bq27_1 == 2), 'bq28'] = 1

#   bq1_list = train_2020.bq1.unique()
#   for bq1 in bq1_list:
#     train_2020.loc[lambda x : (x.bq1 == bq1) & (x.bq30_1.isnull()), 'bq30_1'] = round(train_2020.loc[lambda x: (x.bq1 == bq1) & (x.bq30_1.notnull())].bq30_1.apply(lambda x: int(x)).mean())
#     test_2020.loc[lambda x : (x.bq1 == bq1) & (x.bq30_1.isnull()), 'bq30_1'] = round(test_2020.loc[lambda x: (x.bq1 == bq1) & (x.bq30_1.notnull())].bq30_1.apply(lambda x: int(x)).mean())
    
#     train_2020.loc[lambda x : (x.bq1 == bq1) & (x.bq30_2.isnull()), 'bq30_2'] = round(train_2020.loc[lambda x: (x.bq1 == bq1) & (x.bq30_2.notnull())].bq30_2.apply(lambda x: int(x)).mean())
#     test_2020.loc[lambda x : (x.bq1 == bq1) & (x.bq30_2.isnull()), 'bq30_2'] = round(test_2020.loc[lambda x: (x.bq1 == bq1) & (x.bq30_2.notnull())].bq30_2.apply(lambda x: int(x)).mean())
    
#     try:
#       train_2020.loc[lambda x : (x.bq1 == bq1) & (x.bq30_3.isnull()), 'bq30_3'] = round(train_2020.loc[lambda x: (x.bq1 == bq1) & (x.bq30_3.notnull())].bq30_3.apply(lambda x: int(x)).mean())
#       test_2020.loc[lambda x : (x.bq1 == bq1) & (x.bq30_3.isnull()), 'bq30_3'] = round(test_2020.loc[lambda x: (x.bq1 == bq1) & (x.bq30_3.notnull())].bq30_3.apply(lambda x: int(x)).mean())
#     except:
#       train_2020.loc[lambda x : (x.bq1 == bq1) & (x.bq30_3.isnull()), 'bq30_3'] = round(train_2020.loc[lambda x: (x.bq1 == bq1) & (x.bq30_2.notnull())].bq30_2.apply(lambda x: int(x)).mean())
#       test_2020.loc[lambda x : (x.bq1 == bq1) & (x.bq30_3.isnull()), 'bq30_3'] = round(test_2020.loc[lambda x: (x.bq1 == bq1) & (x.bq30_2.notnull())].bq30_2.apply(lambda x: int(x)).mean())
      
#   return (train_2020, test_2020)

# train[3], test[3] = preprocessing_2020(train[3], test[3])

In [ ]:
from sklearn.preprocessing import LabelEncoder

def label_encoding(data):
    years = ['2017', '2018', '2019', '2020']
    year_encoder = {}
    for year, df in zip(years, data):
        print(year)
        encoders = {}
        for col in df.columns:
            # if (year == '2017') and (col == 'ID' or col == 'bq37' or col == 'bq41_1' or col == 'bq41_2' or col == 'bq41_3'):
            #     df[col] = df[col].fillna(-1)
            #     df[col] = df[col].map(float)
            #     df[col] = df[col].map(int)
            #     continue
            # elif (year == '2018') and (col == 'bq36' or col == 'bq40' or col == 'bq41_1' or col == 'bq41_2' or col == 'bq41_3'):
            #     print(col)
            #     df[col] = df[col].fillna(-1)
            #     df[col] = df[col].map(float)
            #     df[col] = df[col].map(int)
            #     continue
            # elif (year == '2019') and (col == 'bq26' or col == 'bq31_1' or col == 'bq31_1' or col == 'bq31_1' or col == 'bq30'):
            #     df[col] = df[col].fillna(-1)
            #     df[col] = df[col].map(float)
            #     df[col] = df[col].map(int)
            #     continue
            # elif (year == '2020') and (col == 'bq25' or col == 'bq30_1' or col == 'bq30_1' or col == 'bq30_1' or col == 'bq29'):
            #     df[col] = df[col].fillna(-1)
            #     df[col] = df[col].map(float)
            #     df[col] = df[col].map(int)
            #     continue
            try:
                df[col] = df[col].fillna(-1)
                df[col] = df[col].map(float)
                df[col] = df[col].map(int)
            except:
                encoder = LabelEncoder()
                df[col] = df[col].map(str)
                df[col] = encoder.fit_transform(df[col])
                encoders[col] = encoder

        year_encoder[year] = encoders
    return (data, year_encoder)

In [ ]:
train, year_encoder = label_encoding(train)

In [ ]:
test, test_year_encoder = label_encoding(test)
# # test = [df.filter(regex='^(?!bq)') for df in test]
years = ['2017', '2018', '2019', '2020']

x_test = {}
for year, df in zip(years, test):
    x_test[year] = df.iloc[:, 1:]

In [ ]:
# train = [df.filter(regex='^(?!bq)') for df in train]

In [ ]:
x = {}
y = {}

for year, df in zip(years, train):
    x[year] = df.iloc[:, 1:-1]
    x[year] = x[year].fillna(-1)
    y[year] = df.iloc[:, -1]

In [ ]:
from sklearn.model_selection import train_test_split

x_train = {}
y_train = {}
x_valid = {}
y_valid = {}
for year in years:
  x_train[year], x_valid[year], y_train[year], y_valid[year] = train_test_split(x[year], y[year], test_size=0.1, shuffle=True, random_state=42)

In [ ]:
import tensorflow as tf
from tensorflow.keras import backend as K
from tensorflow import argmax as tf_argmax
from sklearn.metrics import f1_score 

def f1(y_true, y_pred):
    y_true = tf_argmax(y_true, axis=1)
    y_pred = tf_argmax(y_pred, axis=1)
    y_pred = K.round(y_pred)
    tp = K.sum(K.cast(y_true*y_pred, 'float'), axis=0)
    # tn = K.sum(K.cast((1-y_true)*(1-y_pred), 'float'), axis=0)
    fp = K.sum(K.cast((1-y_true)*y_pred, 'float'), axis=0)
    fn = K.sum(K.cast(y_true*(1-y_pred), 'float'), axis=0)

    p = tp / (tp + fp + K.epsilon())
    r = tp / (tp + fn + K.epsilon())

    f1 = 2*p*r / (p+r+K.epsilon())
    f1 = tf.where(tf.math.is_nan(f1), tf.zeros_like(f1), f1)
    return K.mean(f1)

def f1_loss(y_true, y_pred):
    
    tp = K.sum(K.cast(y_true*y_pred, 'float'), axis=0)
    tn = K.sum(K.cast((1-y_true)*(1-y_pred), 'float'), axis=0)
    fp = K.sum(K.cast((1-y_true)*y_pred, 'float'), axis=0)
    fn = K.sum(K.cast(y_true*(1-y_pred), 'float'), axis=0)

    p = tp / (tp + fp + K.epsilon())
    r = tp / (tp + fn + K.epsilon())

    f1 = 2*p*r / (p+r+K.epsilon())
    f1 = tf.where(tf.is_nan(f1), tf.zeros_like(f1), f1)
    return 1 - K.mean(f1)

def create_model(label_num):
  model = tf.keras.Sequential([
      tf.keras.layers.Dense(2056, activation='relu'),                         
      tf.keras.layers.Dense(1028, activation='relu'),
      tf.keras.layers.Dense(label_num)
  ])
  model.compile(optimizer='adam',
                loss='categorical_crossentropy',
                metrics=f1)
  return model

tf_result = []
for year in years:
  y = pd.get_dummies(y_train[year], dummy_na=True).applymap(float)

  model = create_model(len(y.columns))
  model.fit(x_train[year], y, epochs=10)
  tf_result.append({'columns':y.columns ,'result':model.predict(x_test[year])})

result2 = [result['columns'][result['result'].argmax(axis=1)] for result in tf_result]
result2 = np.concatenate(result2)
result2

In [ ]:
from sklearn.model_selection import cross_val_score
from catboost import CatBoostClassifier
# score = []
cat_result = []

for year in years:
  print(y[year])
  model = CatBoostClassifier(iterations=10000, verbose=100, task_type='GPU')
  model.fit(x[year], y[year]) #, eval_set=(x_valid[year], y_valid[year]), early_stopping_rounds=10, use_best_model=True 
  cat_result = model.predict(x_test[year])
  cat_result = [result.item() for result in cat_result]
  pd.DataFrame(cat_result).to_csv(f"save3{year}.csv", index=0)
# model = CatBoostClassifier(iterations=100, verbose=100, learning_rate=0.1, task_type='GPU')
# for year in tqdm(years):
#     score.append(cross_val_score(model,x_train[year], y_train[year], scoring='f1_macro', cv=2, fit_params={'eval_set':(x_valid[year], y_valid[year]),'early_stopping_rounds':3,'use_best_model':True}).mean())
# score

In [ ]:
result = [value.item() for value in cat_result]

In [ ]:
result3 = pd.concat([pd.read_csv("save32017.csv"), pd.read_csv("save32018.csv"), pd.read_csv("save32019.csv"), pd.read_csv("save32020.csv")])
temp = pd.read_csv(base_path + "sample_submission.csv")
temp.iloc[:, 1] = np.array(result3['0'])
temp.to_csv("save_result4.csv", index=0)

In [ ]:
sub = pd.read_csv(base_path + "sample_submission.csv")
sub['knowcode'] = result
sub.to_csv("job_recommend14.csv", index=0)

In [ ]:
"""
[0.47636716092615466,
 0.5693723592134896,
 0.5330464383160273,
 0.5134130427012354] all 300

 [0.4490208199966405,
 0.5693723592134896,
 0.5330464383160273,
 0.5134130427012354] 
"""